In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from utils.preprocessing import preprocess_input

In [2]:
# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Load trained model
model = load_model("fer2013_mini_XCEPTION.119-0.65.hdf5", compile=False)

# Emotion labels and colors
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotion_colors = {
    'Angry': (0, 0, 255),
    'Disgust': (0, 128, 0),
    'Fear': (128, 0, 128),
    'Happy': (0, 255, 255),
    'Sad': (255, 0, 0),
    'Surprise': (255, 165, 0),
    'Neutral': (128, 128, 128)
}

In [3]:
# Start video capture
cap = cv2.VideoCapture(0)
print("🟢 Starting Emotion Recognition. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        face = np.expand_dims(face, axis=-1)

        preds = model.predict(face, verbose=0)
        emotion_idx = np.argmax(preds)
        emotion = emotion_labels[emotion_idx]
        color = emotion_colors[emotion]

        # Draw box and label
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"{emotion}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("AI Mood Buddy - Real-Time FER", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

🟢 Starting Emotion Recognition. Press 'q' to quit.
